<a href="https://colab.research.google.com/github/kartik1601/Lane-Detection-OpenCV/blob/main/Lane_Detection_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot
from google.colab.patches import cv2_imshow
from moviepy import editor
import moviepy

In [ ]:
# Driver Function
def process_video(input_video,output_video):
  output_dir = os.path.dirname(output_video)
  if not os.path.exists(output_dir):
        os.makedirs(output_dir)

  cap = cv2.VideoCapture(input_video)
  if not cap.isOpened():
      print("Error: Could not open video.")
      return

  fps = cap.get(cv2.CAP_PROP_FPS)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')

  out = cv2.VideoWriter(output_video, fourcc, fps, (width, height), True)

  while True:
      ret, frame = cap.read()
      if not ret:
          break

      processed_frame = frame_processor(frame)
      out.write(processed_frame)

  cap.release()
  out.release()
  cv2.destroyAllWindows()

In [86]:
# Frame processing function
def frame_processor(image):
  # Converting the RGB into Grayscale mode
  # BGR if input image is taken from cv2
  gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  # Guassian Blur
  # kernel size = 5
  k_size = 5
  blur_image = cv2.GaussianBlur(gray_image, (k_size,k_size) , 0)

  # Canny edge Detection
  low_threshold = 150
  high_threshold = 250

  edges = cv2.Canny(blur_image,low_threshold,high_threshold)

  # Region selection
  marked_region = region_selection(edges)

  # Hough Transform
  hough_image = hough_transform(marked_region)

  # Draw the result
  final_image = draw_lane_lines(image,lane_lines(image,hough_image))
  return final_image

In [ ]:
# Selecting the necessary region
def region_selection(image):
  mask_img = np.zeros_like(image)

  # If more than one channel is present in image
  # Defining color of masked polygon
  if len(image.shape) > 2:
    channel = image.shape[2] # h,w,c - 0,1,2
    ignore_mask_color = (255,) * channel

  else:
    ignore_mask_color = 255

  rows, cols = image.shape[:2]
  bottom_left = [cols*0.1, rows*0.95]
  top_left = [cols*0.4, rows*0.6]
  bottom_right = [cols*0.9, rows*0.95]
  top_right = [cols*0.6, rows*0.6]

  polygon_vertices = np.array([bottom_left,top_left,top_right,bottom_right],dtype=np.int32)

  cv2.fillPoly(mask_img, [polygon_vertices],ignore_mask_color)

  masked_img = cv2.bitwise_and(image,mask_img)

  return masked_img

In [90]:
# Hough transform for identifying straight lines

def hough_transform(image):
  # Distance Resolution
  rho = 1
  # Angle resolution
  theta = np.pi/180
  # threshold
  threshold = 12
  # Minimum length of line
  minLineLength = 16
  # Mximum allowed gap
  maxLineGap = 500

  return cv2.HoughLinesP(image,rho=rho,theta=theta,threshold=threshold,minLineLength=minLineLength,maxLineGap=maxLineGap)

In [72]:
# Plotting hough lines on our input frame
# Slope intercept - for calculating slope and intercept
# Pixel points - using line equation, plotting their x,y coords
# Lane Lines - for calculating above two for left and right lanes

def avg_slope_intercept(lines):

  left_lines    = [] #(slope, intercept)
  left_weights  = [] #(length,)
  right_lines   = [] #(slope, intercept)
  right_weights = [] #(length,)

  for line in lines:
    for x1,y1,x2,y2 in line:
      if x1==x2:
        continue

      slope = (y2-y1)/(x2-x1)
      intercept = y2 - (slope*x2)
      length = np.sqrt(((y2-y1)**2) + ((x2-x1)**2))

      if slope<0:
        left_lines.append((slope,intercept))
        left_weights.append((length))
      else:
        right_lines.append((slope,intercept))
        right_weights.append((length))

  left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
  right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None

  return left_lane, right_lane

def pixel_points(y1,y2,line):

  if line is None:
    return None

  slope, intercept = line
  x1 = int((y1 - intercept)/slope)
  x2 = int((y2 - intercept)/slope)
  y1 = int(y1)
  y2 = int(y2)
  return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
  # Image = input image
  # lines = lines after hough transform
  left_lane, right_lane = avg_slope_intercept(lines)

  y1 = image.shape[0]
  y2 = y1 * 0.6

  left_line = pixel_points(y1,y2,left_lane)
  right_line = pixel_points(y1,y2,right_lane)

  return left_line, right_line

def draw_lane_lines(image,lines,color=[0,255,0],thickness=6):
  line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

  for line in lines:
    if line is not None:
      cv2.line(line_image,*line,color,thickness)

  return cv2.addWeighted(image,1.0,line_image,1.0,0.0)

In [ ]:
!git clone https://github.com/udacity/CarND-LaneLines-P1.git

In [94]:
input_path = '/content/CarND-LaneLines-P1/test_videos/test_video.mp4'
output_path = '/content/CarND-LaneLines-P1/test_videos_output/output6.mp4'

In [95]:
process_video(input_path,output_path)